<a href="https://colab.research.google.com/github/rumman-adnan/Assignment-Sensors-Data/blob/main/sensors_data_analytics_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [4]:
train_data = np.load('/content/drive/MyDrive/Python work/sim_data/simu_20000_0.1_90_140_train.npy')
test_data = np.load('/content/drive/MyDrive/Python work/sim_data/simu_10000_0.1_141_178_test.npy')

print("The shape of trained data is: ",train_data.shape)
print("The shape of tested data is: ",test_data.shape)

The shape of trained data is:  (20000, 1006)
The shape of tested data is:  (10000, 1006)


In [5]:
# Extraction of features and targets

# Training features and targets
X_train_lstm = train_data[:, :1000]
Y_train_lstm = train_data[:, -2:]

# Testing features and targets
X_test_lstm = test_data[:, :1000]
Y_test_lstm = test_data[:, -2:]

# Reshaping data for LSTM
X_train_lstm = X_train_lstm.reshape(X_train_lstm.shape[0], X_train_lstm.shape[1], 1)
X_test_lstm = X_test_lstm.reshape(X_test_lstm.shape[0], X_test_lstm.shape[1], 1)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_absolute_error


# #
# X_train_lstm = X_train_lstm.reshape(X_train_lstm.shape[0], X_train_lstm.shape[1], 1)
# X_test_lstm = X_test_lstm.reshape(X_test_lstm.shape[0], X_test_lstm.shape[1], 1)


# # Reshaping data for LSTM, Data shape is already suitable for LSTM: (batch_size, timesteps, features)
# X_train_lstm = X_train_lstm.reshape(X_train_lstm.shape[0], X_train_lstm.shape[1], 1)
# X_test_lstm = X_test_lstm.reshape(X_test_lstm.shape[0], X_test_lstm.shape[1], 1)

# Model Creation
model = Sequential()

# Adding LSTM layer optimized for cuDNN
model.add(LSTM(50, activation='tanh', recurrent_activation='sigmoid', input_shape=(X_train_lstm.shape[1], 1)))

# Dense layers for regression
model.add(Dense(50, activation='relu'))
model.add(Dense(2))  # Two output units for S and D

# Define optimizer and loss
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.MeanSquaredError()

# Custom training loop
epochs = 500  # Maximum epochs
batch_size = 32
desired_mae = 3
steps_per_epoch = len(X_train_lstm) // batch_size

for epoch in range(epochs):
    print("\nEpoch {}/{}".format(epoch+1, epochs))

    for step in range(steps_per_epoch):
        # Create a batch of data
        X_batch = X_train_lstm[step*batch_size:(step+1)*batch_size]
        Y_batch = Y_train_lstm[step*batch_size:(step+1)*batch_size]

        # Forward pass
        with tf.GradientTape() as tape:
            predictions = model(X_batch, training=True)
            loss_value = loss_fn(Y_batch, predictions)

        # Backward pass
        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Evaluate after each epoch
    predictions = model(X_test_lstm)
    mae_scores = mean_absolute_error(Y_test_lstm, predictions, multioutput='raw_values')
    print("MAE after epoch {}: {}".format(epoch+1, mae_scores))

    # If desired MAE is achieved, stop training
    if mae_scores[0] <= desired_mae and mae_scores[1] <= desired_mae:
        print("Desired MAE achieved. Stopping training.")
        break


### Apply Min Max Scaler

In [6]:
# Initializing the scalers
feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

# Scaling features
X_train_scaled = feature_scaler.fit_transform(X_train_lstm.reshape(X_train_lstm.shape[0], X_train_lstm.shape[1]))
X_test_scaled = feature_scaler.transform(X_test_lstm.reshape(X_test_lstm.shape[0], X_test_lstm.shape[1]))
X_train_scaled = X_train_scaled.reshape(X_train_lstm.shape[0], X_train_lstm.shape[1], 1)
X_test_scaled = X_test_scaled.reshape(X_test_lstm.shape[0], X_test_lstm.shape[1], 1)

# Scaling targets
Y_train_scaled = target_scaler.fit_transform(Y_train_lstm)
Y_test_scaled = target_scaler.transform(Y_test_lstm)

X_train_scaled.shape, X_test_scaled.shape, Y_train_scaled.shape, Y_test_scaled.shape


((20000, 1000, 1), (10000, 1000, 1), (20000, 2), (10000, 2))

### Train LSTM Model

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_absolute_error

# Define the LSTM model
model_lstm = Sequential()
model_lstm.add(LSTM(50, activation='tanh', recurrent_activation='sigmoid', input_shape=(X_train_scaled.shape[1], 1)))
model_lstm.add(Dense(50, activation='relu'))
model_lstm.add(Dense(2))  # Two output units for S and D

# Compiling the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_lstm.compile(optimizer=optimizer, loss='mean_squared_error')

# Training the LSTM model
desired_mae = 3
best_mae = float('inf')
patience = 10  # Number of epochs to wait if no improvement before stopping
wait = 0

for epoch in range(500):  # Maximum epochs
    print(f"\nEpoch {epoch+1}/500")

    # Train the model
    model_lstm.fit(X_train_scaled, Y_train_scaled, batch_size=32, epochs=1, verbose=1, validation_data=(X_test_scaled, Y_test_scaled))

    # Evaluate the model's performance on the test set
    predictions = model_lstm.predict(X_test_scaled)
    mae_scores = mean_absolute_error(Y_test_scaled, predictions, multioutput='raw_values')
    print(f"Mean Absolute Error for S: {mae_scores[0]}, for D: {mae_scores[1]}")

    # Early stopping based on desired MAE
    if mae_scores[0] <= desired_mae and mae_scores[1] <= desired_mae:
        print("Desired MAE achieved. Stopping training.")
        break

    # Implementing patience for early stopping
    if mae_scores[0] < best_mae or mae_scores[1] < best_mae:
        best_mae = min(mae_scores)
        wait = 0
    else:
        wait += 1
        if wait >= patience:
            print(f"MAE did not improve for {patience} consecutive epochs. Stopping training.")
            break



Epoch 1/500
313/313 [==============================] - 4s 12ms/step
Mean Absolute Error for S: 0.8776830042492247, for D: 0.2555532025271651
Desired MAE achieved. Stopping training.
